# Data Prep

In [18]:
import pandas as pd
import numpy as np
import os
import csv
import cv2

In [8]:
driving_log = pd.DataFrame.from_csv('sample_data/driving_log.csv')
driving_log.head()

,left,right,steering,throttle,brake,speed
center,,,,,,
IMG/center_2016_12_01_13_30_48_287.jpg,IMG/left_2016_12_01_13_30_48_287.jpg,IMG/right_2016_12_01_13_30_48_287.jpg,0.0,0.0,0.0,22.148290
IMG/center_2016_12_01_13_30_48_404.jpg,IMG/left_2016_12_01_13_30_48_404.jpg,IMG/right_2016_12_01_13_30_48_404.jpg,0.0,0.0,0.0,21.879630
IMG/center_2016_12_01_13_31_12_937.jpg,IMG/left_2016_12_01_13_31_12_937.jpg,IMG/right_2016_12_01_13_31_12_937.jpg,0.0,0.0,0.0,1.453011
IMG/center_2016_12_01_13_31_13_037.jpg,IMG/left_2016_12_01_13_31_13_037.jpg,IMG/right_2016_12_01_13_31_13_037.jpg,0.0,0.0,0.0,1.438419
IMG/center_2016_12_01_13_31_13_177.jpg,IMG/left_2016_12_01_13_31_13_177.jpg,IMG/right_2016_12_01_13_31_13_177.jpg,0.0,0.0,0.0,1.418236


In [12]:
lines = []
with open('sample_data/driving_log.csv') as f:
    reader = csv.reader(f)
    for line in reader:
        lines.append(line)

In [32]:
images = []
measurements = []

for line in lines[1:]:
    fn = line[0] #.split('/')[-1]
    img = cv2.imread('sample_data/'+fn)
    images.append(img)
    measurements.append(float(line[3]))    


In [33]:
X_train = np.array(images)
y_train = np.array(measurements)

In [34]:
X_train.shape

(8036, 160, 320, 3)

---
# Model Architecture

In [42]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Lambda

In [36]:
EPOCHS = 7

In [43]:
model = Sequential()

# normalize
model.add(Lambda(lambda x: x/255.0 - 0.5, input_shape=(160, 320, 3)))

model.add(Flatten())
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')

In [44]:
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=EPOCHS, verbose=1)

Train on 6428 samples, validate on 1608 samples
Epoch 1/7
6428/6428 [==============================] - 9s - loss: 1.9677 - val_loss: 0.4604
Epoch 2/7
6428/6428 [==============================] - 9s - loss: 2.1805 - val_loss: 2.0877
Epoch 3/7
6428/6428 [==============================] - 9s - loss: 4.1312 - val_loss: 2.3634
Epoch 4/7
6428/6428 [==============================] - 9s - loss: 5.6338 - val_loss: 11.4817
Epoch 5/7
6428/6428 [==============================] - 9s - loss: 4.4341 - val_loss: 5.9602
Epoch 6/7
6428/6428 [==============================] - 9s - loss: 3.2845 - val_loss: 3.1250
Epoch 7/7
6428/6428 [==============================] - 9s - loss: 2.3186 - val_loss: 1.3500


In [41]:
model.save('models/momo.h5')